# Graph Data structures

In [ ]:
# create node without calculating early start and early finish
"""CREATE (activity:ACTIVITY {name: $name, description: $description, duration: $duration})
                    WITH activity, $predecessors as batch
                    UNWIND batch as dependency
                    MATCH (predecessor:ACTIVITY)
                    WHERE ID(predecessor) = dependency.activity.id
                    CREATE (activity)<-[:DEPENDS_ON {type: dependency.type, duration: dependency.duration}]-(predecessor)"""

# TODO: create an activity and calculate earlyStart and early finish. User will see in real time taken to finish project 

Topological ordering

In [ ]:
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COUNT(predecessor) AS incomingDependencies
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, incomingDependencies, COALESCE(MAX(LENGTH(path)), 0) AS depth
RETURN 
    activity.name AS activityName, 
    activity.description AS description, 
    activity.duration AS duration, 
    depth, 
    incomingDependencies
ORDER BY depth DESC, incomingDependencies ASC


Early start and early finish topological ordering

In [ ]:
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COLLECT(predecessor) AS predecessors
ORDER BY activity.depth ASC
WITH activity,
    REDUCE(maxFinish = 0, pred IN predecessors | 
        CASE 
            WHEN pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish THEN pred.earlyFinish 
            ELSE maxFinish 
        END) AS earlyStart
SET activity.earlyStart = COALESCE(earlyStart, 0),
    activity.earlyFinish = COALESCE(earlyStart, 0) + activity.duration
RETURN activity.name AS activityName, activity.earlyStart, activity.earlyFinish
ORDER BY activity.depth ASC

In [1]:
"""
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COUNT(predecessor) AS incomingDependencies
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, incomingDependencies, COALESCE(MAX(LENGTH(path)), 0) AS depth
RETURN 
    activity.name AS activityName, 
    activity.description AS description, 
    activity.duration AS duration, 
    depth, 
    incomingDependencies
ORDER BY depth DESC, incomingDependencies ASC

MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)
WITH activity, COLLECT(predecessor) AS predecessors
ORDER BY activity.depth ASC
WITH activity,
    REDUCE(maxFinish = 0, pred IN predecessors | 
        CASE 
            WHEN pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish THEN pred.earlyFinish 
            ELSE maxFinish 
        END) AS earlyStart
SET activity.earlyStart = COALESCE(earlyStart, 0),
    activity.earlyFinish = COALESCE(earlyStart, 0) + activity.duration
RETURN activity.name AS activityName, activity.earlyStart, activity.earlyFinish
ORDER BY activity.depth ASC
"""

SyntaxError: invalid syntax (175008303.py, line 1)

In [ ]:
"""
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)    
WITH activity, predecessor, COUNT(predecessor) AS incomingDependencies     

OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY) 
WITH activity, predecessor, incomingDependencies, COALESCE(MAX(LENGTH(path)), 0) AS depth 
ORDER BY depth ASC, incomingDependencies ASC   // activity with longest chain comes first
WITH activity, COLLECT(predecessor) AS predecessors                 // group all predecessor nodes into a list
WITH activity,
    REDUCE(maxFinish = 0, pred IN predecessors |                    //  Iterate over each predecessor for forward pass take max of predecessors early finish 
        CASE 
            WHEN pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish THEN pred.earlyFinish  // get max early finish of predecessors
            ELSE maxFinish 
        END) AS earlyStart                                          // the maxfinish in the predecessors will be the early start of current node  or 0 if no predecessors acc maxFinish = 0                     
SET activity += {earlyStart: COALESCE(earlyStart, 0),
    earlyFinish: COALESCE(earlyStart, 0) + activity.duration} 
RETURN activity.name AS activityName, activity.earlyStart, activity.earlyFinish
ORDER BY activity.depth ASC;"""

In [ ]:
"""
// initialize earlystart and early finish of the last node
MATCH (activity:ACTIVITY)
WHERE NOT (activity)-[:DEPENDS_ON]->()  // Find node with no outgoing relationship
SET activity.lateFinish = activity.earlyFinish,
    activity.lateStart = activity.lateFinish - activity.duration;

// Propagate lateStart and lateFinish values backward
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY) // Match successors
WITH activity, 
     COLLECT(successor) AS successors                         // Group successors for each node
ORDER BY activity.depth DESC                                  // Process nodes in reverse order of depth
WITH activity,
     REDUCE(minStart = activity.lateFinish, succ IN successors | 
         CASE 
             WHEN succ.lateStart IS NOT NULL AND succ.lateStart < minStart THEN succ.lateStart
             ELSE minStart
         END) AS lateFinish                                 // Calculate lateFinish as min(lateStart) of successors
SET activity.lateFinish = COALESCE(lateFinish, activity.earlyFinish),
    activity.lateStart = activity.lateFinish - activity.duration;

// Step 3: Return Results
MATCH (activity:ACTIVITY)
RETURN activity.name AS activityName, activity.lateStart, activity.lateFinish
ORDER BY activity.depth DESC;
"""

## topological order
Only works for Directed Acyclic Graphs

In [7]:
"""// Match all activities and their predecessors
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)

// Collect predecessors for each activity
WITH activity, COLLECT(predecessor.name) AS predecessors

// Find the depth of each activity for topological order
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, predecessors, 
     COALESCE(MAX(LENGTH(path)), 0) AS depth, 
     SIZE(predecessors) AS incomingDependencies

// Order activities by depth (descending) and by number of incoming dependencies (ascending)
ORDER BY depth DESC, incomingDependencies ASC 

// Return the results with the predecessors as a list
RETURN 
    activity.name AS name, 
    depth, 
    incomingDependencies, 
    predecessors AS predecessorNames
"""

'// Match all activities and their predecessors\nMATCH (activity:ACTIVITY)\nOPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)\n\n// Collect predecessors for each activity\nWITH activity, COLLECT(predecessor.name) AS predecessors\n\n// Find the depth of each activity for topological order\nOPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)\nWITH activity, predecessors, \n     COALESCE(MAX(LENGTH(path)), 0) AS depth, \n     SIZE(predecessors) AS incomingDependencies\n\n// Order activities by depth (descending) and by number of incoming dependencies (ascending)\nORDER BY depth DESC, incomingDependencies ASC \n\n// Return the results with the predecessors as a list\nRETURN \n    activity.name AS name, \n    depth, \n    incomingDependencies, \n    predecessors AS predecessorNames\n'

### Forward pass
It does calculate the correct values but one has to run the query a couple of time for it to calculate the correct values. Reason is Neo4j is propagating stale values for earlyStart and earlyFinish to the next node.

In [8]:
"""// Match all activities and their predecessors
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)

// Collect predecessors for each activity
WITH activity, COLLECT(predecessor) AS predecessors

// Find the depth of each activity for topological order
OPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)
WITH activity, predecessors, 
     COALESCE(MAX(LENGTH(path)), 0) AS depth, 
     SIZE(predecessors) AS incomingDependencies

// Order activities by depth (descending) and by number of incoming dependencies (ascending) topological order
ORDER BY depth DESC, incomingDependencies ASC 

// Calculate earlyStart and earlyFinish for each activity
WITH COLLECT([activity, predecessors]) AS activityPredecessorsList
UNWIND activityPredecessorsList AS activityPredecessorsTuple
WITH activityPredecessorsTuple[0] AS currentActivity, 
     REDUCE(
         maxFinish = 0, 
         pred IN activityPredecessorsTuple[1] |                    
         CASE 
             WHEN pred IS NOT NULL AND pred.earlyFinish IS NOT NULL AND pred.earlyFinish > maxFinish 
             THEN pred.earlyFinish  
             ELSE maxFinish 
         END
     ) AS earlyStart
SET currentActivity += {
    earlyStart: COALESCE(earlyStart, 0), 
    earlyFinish: COALESCE(earlyStart, 0) + currentActivity.duration
}
RETURN currentActivity.name AS activityName, 
       currentActivity.earlyStart AS earlyStart, 
       currentActivity.earlyFinish AS earlyFinish
ORDER BY currentActivity.depth ASC
"""

'// Match all activities and their predecessors\nMATCH (activity:ACTIVITY)\nOPTIONAL MATCH (activity)<-[:DEPENDS_ON]-(predecessor:ACTIVITY)\n\n// Collect predecessors for each activity\nWITH activity, COLLECT(predecessor) AS predecessors\n\n// Find the depth of each activity for topological order\nOPTIONAL MATCH path = (activity)-[:DEPENDS_ON*]->(dependent:ACTIVITY)\nWITH activity, predecessors, \n     COALESCE(MAX(LENGTH(path)), 0) AS depth, \n     SIZE(predecessors) AS incomingDependencies\n\n// Order activities by depth (descending) and by number of incoming dependencies (ascending) topological order\nORDER BY depth DESC, incomingDependencies ASC \n\n// Calculate earlyStart and earlyFinish for each activity\nWITH COLLECT([activity, predecessors]) AS activityPredecessorsList\nUNWIND activityPredecessorsList AS activityPredecessorsTuple\nWITH activityPredecessorsTuple[0] AS currentActivity, \n     REDUCE(\n         maxFinish = 0, \n         pred IN activityPredecessorsTuple[1] |      

### backward pass

In [9]:
"""
// match all activities and their successors
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY)

// collect successors for each activity
WITH activity, COLLECT(successor) AS successors

// find the depth of each activity for topological order
OPTIONAL MATCH path = (activity)<-[:DEPENDS_ON*]-(dependent:ACTIVITY)
WITH activity, successors, 
    COALESCE(MAX(LENGTH(path)), 0) AS depth, 
    SIZE(successors) AS outgoingDependencies

// order activities by depth (descending) and by number of outgoing dependencies (ascending) topological order
ORDER BY depth DESC, outgoingDependencies ASC 

// calculate lateStart and lateFinish for each activity
WITH COLLECT([activity, successors]) AS activitySuccessorsList
UNWIND activitySuccessorsList AS activitySuccessorsTuple
WITH activitySuccessorsTuple[0] AS currentActivity, activitySuccessorsTuple
    WITH currentActivity, activitySuccessorsTuple, REDUCE(
        minStart = currentActivity.earlyFinish, 
        succ IN activitySuccessorsTuple[1] |                    
        CASE 
            WHEN succ IS NOT NULL AND succ.lateStart IS NOT NULL AND succ.lateStart < minStart 
            THEN succ.lateStart  
            ELSE minStart 
        END
    ) AS lateFinish

// update lateStart and lateFinish for each activity
SET currentActivity += {
    lateFinish: COALESCE(lateFinish, currentActivity.earlyFinish), 
    lateStart: COALESCE(lateFinish, currentActivity.earlyFinish) - currentActivity.duration
}

// return the results
RETURN currentActivity.name AS activityName, 
    currentActivity.lateStart AS lateStart, 
    currentActivity.lateFinish AS lateFinish
ORDER BY currentActivity.depth DESC"""

'\n// match all activities and their successors\nMATCH (activity:ACTIVITY)\nOPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY)\n\n// collect successors for each activity\nWITH activity, COLLECT(successor) AS successors\n\n// find the depth of each activity for topological order\nOPTIONAL MATCH path = (activity)<-[:DEPENDS_ON*]-(dependent:ACTIVITY)\nWITH activity, successors, \n    COALESCE(MAX(LENGTH(path)), 0) AS depth, \n    SIZE(successors) AS outgoingDependencies\n\n// order activities by depth (descending) and by number of outgoing dependencies (ascending) topological order\nORDER BY depth DESC, outgoingDependencies ASC \n\n// calculate lateStart and lateFinish for each activity\nWITH COLLECT([activity, successors]) AS activitySuccessorsList\nUNWIND activitySuccessorsList AS activitySuccessorsTuple\nWITH activitySuccessorsTuple[0] AS currentActivity, activitySuccessorsTuple\n    WITH currentActivity, activitySuccessorsTuple, REDUCE(\n        minStart = currentActivity.e

## Calculating Slack
Free slack
Total slack

In [ ]:
"""
MATCH (activity:ACTIVITY)
OPTIONAL MATCH (activity)-[:DEPENDS_ON]->(successor:ACTIVITY)

WITH activity, COLLECT(successor) AS successors

OPTIONAL MATCH path = (activity)<-[:DEPENDS_ON*]-(dependent:ACTIVITY)
WITH activity, successors, 
    COALESCE(MAX(LENGTH(path)), 0) AS depth, 
    SIZE(successors) AS outgoingDependencies

ORDER BY depth DESC, outgoingDependencies ASC 

WITH COLLECT([activity, successors]) AS activitySuccessorsList
UNWIND activitySuccessorsList AS activitySuccessorsTuple
WITH activitySuccessorsTuple[0] AS currentActivity, activitySuccessorsTuple
    WITH currentActivity, activitySuccessorsTuple, REDUCE(
        minSuccEarlyStart = activitySuccessorsTuple[1][0].earlyStart, 
        succ IN activitySuccessorsTuple[1] |                    
        CASE 
            WHEN succ IS NOT NULL AND succ.earlyStart IS NOT NULL AND succ.earlyStart < minSuccEarlyStart 
            THEN succ.earlyStart  
            ELSE minSuccEarlyStart 
        END
    ) AS minSuccEarlyStart
SET currentActivity += {
    totalFloat: currentActivity.lateFinish - currentActivity.earlyFinish,
    freeFloat: COALESCE(minSuccEarlyStart - currentActivity.earlyStart - currentActivity.duration, 0)
}
RETURN currentActivity.name AS activityName, 
    currentActivity.earlyStart AS earlyStart, 
    currentActivity.earlyFinish AS earlyFinish,
    currentActivity.lateStart AS lateStart,
    currentActivity.lateFinish AS lateFinish,
    currentActivity.totalFloat AS totalFloat,
    currentActivity.freeFloat AS freeFloat
ORDER BY currentActivity.depth DESC"""

### Critical path

In [ ]:
"""
// trace paths between critical activities
OPTIONAL MATCH path = (start:ACTIVITY)-[:DEPENDS_ON*]->(end:ACTIVITY)
WHERE ALL(node IN NODES(path) WHERE node.totalFloat = 0)

//Total duration between the critical activities
WITH path, 
     REDUCE(totalDuration = 0, activity IN NODES(path) | totalDuration + activity.duration) AS pathDuration
ORDER BY pathDuration DESC

// Return the longest path and duration
LIMIT 1
RETURN [activity IN NODES(path) | activity.name] AS criticalPath, pathDuration
"""

In [ ]:
// Create activity
CREATE (activity:ACTIVITY {name: "Foundation Excavation", duration: 10, start_date: "2024-01-01", end_date: "2024-01-10"})

// Create allocation
CREATE (allocation:ALLOCATION {cost: 500, quantity: 3, unit_cost: 150, transport_cost: 50, allocation_date: "2024-01-01", release_date: "2024-01-10"})

// Create resources
CREATE (labor:RESOURCE {type: "labor", skill: "plumbing", rate: 50, availability: 10})
CREATE (equipment:RESOURCE {type: "equipment", name: "Excavator", source: "leasing", rental_cost: 200, transport_cost: 50})
CREATE (material:RESOURCE {type: "material", name: "Concrete", unit_cost: 20, MOQ: 5, availability: 100})

// Link activity to allocation
CREATE (activity)-[:ALLOCATED]->(allocation)

// Link allocation to resources
CREATE (allocation)-[:USES]->(labor)
CREATE (allocation)-[:USES]->(equipment)
CREATE (allocation)-[:USES]->(material)

// Define requirements
CREATE (activity)-[:REQUIRES {quantity: 5}]->(material)
CREATE (activity)-[:REQUIRES {quantity: 1}]->(equipment)
CREATE (activity)-[:REQUIRES {quantity: 3, skill: "plumbing"}]->(labor)


In [ ]:
MATCH (resource:RESOURCE)
WHERE resource.type = "labor" AND resource.skill = "plumbing"
RETURN resource.availability >= 5 AS isAvailable


In [ ]:
MATCH (allocation:ALLOCATION)-[:USES]->(resource:RESOURCE)
RETURN allocation.cost + SUM(resource.transport_cost) AS totalCost


In [ ]:
MATCH (activity:ACTIVITY)-[:REQUIRES {quantity: required}]->(resource:RESOURCE)
WHERE resource.availability < required
RETURN resource.type, required - resource.availability AS deficit


In [ ]:
MATCH (allocation:ALLOCATION)-[:USES]->(resource:RESOURCE)
WHERE allocation.release_date <= date()
SET resource.availability = resource.availability + allocation.quantity


Create labour, equipment and material nodes. Group materials of same type

In [ ]:
# Group resources
"""MATCH (l:Labor {type: 'plumbing'})
CREATE (r:Resource {type: 'labor'})
CREATE (r)-[:INCLUDES]->(l);

MATCH (e:Equipment {name: 'Concrete Mixer'})
CREATE (r:Resource {type: 'equipment'})
CREATE (r)-[:INCLUDES]->(e);

MATCH (m:Material {name: 'Cement'})
CREATE (r:Resource {type: 'material'})
CREATE (r)-[:INCLUDES]->(m);
"""

In [ ]:
# link an activity to resource
"""CREATE (a:Activity {name: 'Foundation', startDate: '2024-01-01', endDate: '2024-01-15'});

MATCH (r:Resource {type: 'labor'}), (a:Activity {name: 'Foundation'})
CREATE (a)-[:REQUIRES]->(r);

MATCH (r:Resource {type: 'equipment'}), (a:Activity {name: 'Foundation'})
CREATE (a)-[:REQUIRES]->(r);
"""

In [ ]:
# allocate resource and update the resource availability
"""MATCH (a:Activity {name: 'Foundation'}), (r:Resource {type: 'labor'})
CREATE (a)-[:ALLOCATED {cost: 400, duration: '8 hours'}]->(r)
SET r.available = false;

MATCH (a:Activity {name: 'Foundation'}), (r:Resource {type: 'material'})
CREATE (a)-[:ALLOCATED {cost: 250, quantity: 50}]->(r)
SET r.quantity = r.quantity - 50;
"""

In [ ]:
# Link materials to supplier
"""CREATE (:Supplier {name: 'Supplier A', contact: 'supplierA@example.com'});

MATCH (m:Material {name: 'Cement'}), (s:Supplier {name: 'Supplier A'})
CREATE (m)-[:SUPPLIED_BY]->(s);
"""

### creating equipments
TODO: equipment has to have an operator so consider if equipment cost is together with the operator or not

In [3]:
"""CREATE (equipment:EQUIPMENT {name: "Concrete mixer", quantity: 3, description: "For mixing and batching concrete", source: "leading", cost: 4000, rate: "per-day", transportCost: 1000})
CREATE (equipment:EQUIPMENT {name: "Poker vibrator", quantity: 3, description: "For compacting concrete", source: "leasing", cost: 2500, rate: "per-day", transportCost: 1000})
CREATE (equipment:EQUIPMENT {name: "Bulldozer", quantity: 1, description: "For lifting waste into tipper lorries", source: "leasing", cost: 12000, rate: "per-trip", transportCost: 0})
CREATE (equipment:EQUIPMENT {name: "Excavator", quantity: 1, description: "For digging", source: "leasing", cost: 18000, rate: "per-trip", transportCost: 0})
CREATE (equipment:EQUIPMENT {name: "Tippers", quantity: 2, description: "Transport and dump waste", source: "leasing", cost: 4000, rate: "per-trip", transportCost: 0})
CREATE (equipment:EQUIPMENT {name: "Lever Hoist", quantity: 2, description: "For lifting items to the top floors of building instead of a crane", source: "leasing", cost: 10000, rate: "per-day", transportCost: 2000})"""

SyntaxError: invalid syntax (112490860.py, line 1)